In [1]:
## MC dropout ## AI-generated

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

2025-12-12 18:39:04.695597: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


`-` Image Dataset

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

In [4]:
def build_mc_dropout_model(dropout_rate=0.25):
    model = keras.Sequential([
        layers.InputLayer(input_shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(dropout_rate),  # Dropout after pooling
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(dropout_rate),  # Dropout after pooling
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(dropout_rate),  # Dropout after dense layer
        layers.Dense(10, activation="softmax"),
    ])
    return model

In [5]:
model = build_mc_dropout_model()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.1)


/root/anaconda3/envs/TF/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1765532432.749297   25076 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79121 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 7g.80gb, pci bus id: 0000:47:00.0, compute capability: 8.0


Epoch 1/5


2025-12-12 18:40:35.273694: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f20cc004880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-12 18:40:35.273718: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB MIG 7g.80gb, Compute Capability 8.0
2025-12-12 18:40:35.311298: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-12 18:40:35.488753: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


 83/422 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5433 - loss: 1.3789

I0000 00:00:1765532438.591713   25495 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


417/422 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7870 - loss: 0.6595

2025-12-12 18:40:39.944710: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1384', 4 bytes spill stores, 4 bytes spill loads



422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.8990 - loss: 0.3213 - val_accuracy: 0.9812 - val_loss: 0.0687
Epoch 2/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9704 - loss: 0.0966 - val_accuracy: 0.9870 - val_loss: 0.0460
Epoch 3/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9779 - loss: 0.0709 - val_accuracy: 0.9875 - val_loss: 0.0426
Epoch 4/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9817 - loss: 0.0598 - val_accuracy: 0.9885 - val_loss: 0.0375
Epoch 5/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9838 - loss: 0.0515 - val_accuracy: 0.9898 - val_loss: 0.0380


In [6]:
sample_indices = np.random.choice(len(x_test), 4, replace=False)
sample_indices = np.array([0,1,2,3])
x_sample = x_test[sample_indices]
y_true_sample = y_test[sample_indices]

In [ ]:
def mc_dropout_predict(model, x, n_samples=500):
    predictions = []
    for _ in range(n_samples):
        # Crucially, set training=True to keep dropout active during inference
        output = model(x, training=True)
        predictions.append(output.numpy())
    predictions = np.array(predictions)
    mean_predictions = predictions.mean(axis=0) ## posterior mean
    posteriors = predictions
    return mean_predictions, posteriors # posterior: n_samples*N*10

def MI(posteriors):
    pp = posteriors.mean(axis=0)
    v1 = - np.sum(pp * np.log(pp),axis=1)   ## MI first term (entropy)
    tmp = - np.sum(posteriors * np.log(posteriors), axis=2)
    v2 = np.mean(tmp, axis=0)   ## MI second term (posterior entropy mean)
    return v1 - v2

In [8]:
mean_preds, posteriors = mc_dropout_predict(model, x_sample)
print(posteriors.shape)
uncertainties = MI(posteriors)
# Display results for the sampled images
for i in range(len(x_sample)):
    predicted_class = np.argmax(mean_preds[i])
    true_class = y_true_sample[i]
    pred_uncertainty = uncertainties[i] # Uncertainty for the predicted class
    print(f"Image {i+1}: True Class: {true_class}, Predicted Class: {predicted_class}, MI: {pred_uncertainty:.4f}")

(500, 4, 10)
Image 1: True Class: 7, Predicted Class: 7, MI: 0.0010
Image 2: True Class: 2, Predicted Class: 2, MI: 0.0035
Image 3: True Class: 1, Predicted Class: 1, MI: 0.0010
Image 4: True Class: 0, Predicted Class: 0, MI: 0.0090


* Dropout Rate: 0.5

```raw
Image 1: True Class: 7, Predicted Class: 7, MI: 0.0041
Image 2: True Class: 2, Predicted Class: 2, MI: 0.0236
Image 3: True Class: 1, Predicted Class: 1, MI: 0.0138
Image 4: True Class: 0, Predicted Class: 0, MI: 0.0123
```

> MI가 클수록 불확실성이 높음. Dropout Rate에 따라 MI의 값이 변화함
>
> Dropout Rate를 줄이면 MI가 줄어듦.